# Lab 5: Iterative Methods for Linear Systems

### Topics

- **Mathematics:** solving $3\times 3$ linear systems by Jacobi and Gauss–Seidel methods.
- **Python:** writing code for iterative methods for solving linear systems; using flags to indicate convergence.

In [ ]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt

## Jacobi method

Consider the linear system $A\underline{\mathbf{x}}=\underline{\mathbf{b}}$ where

$$
  A = \begin{bmatrix}
        4 & \phantom{-}1 & \phantom{-}1 \\
        1 & \phantom{-}2 & \phantom{-}3 \\
        2 & -1 & -3
      \end{bmatrix}
  \qquad
  \underline{\mathbf{b}} = \begin{bmatrix}
          \phantom{-}5 \\
          \phantom{-}5 \\
          -3
        \end{bmatrix}.
$$

Start by writing down the system of linear equations **on paper**.  Rearrange the equations into a suitable form for the Jacobi method.

Write a function `jacobi` to carry out one step of the Jacobi method for a given initial estimate $\underline{\mathbf{x}}^{\left(0\right)}$.

```
def jacobi(A, b, x):
    ...
    return xnew
```

Test your function using an initial estimate of $\underline{\mathbf{x}}^{\left(0\right)}=(0,0,0)$, checking the output with a hand calculation if necessary.

In [23]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt

def jacobi(A,b,x):
    x0 = ( b[0] - A[0][1]*x[1] - A[0][2]*x[2] ) / A[0][0]
    x1 = ( b[1] - A[1][1]*x[1] - A[1][2]*x[2] ) / A[1][1]
    x2 = ( b[2] - A[2][1]*x[1] - A[2][2]*x[2] ) / A[2][2]
    
    xnew = np.array([x0,x1,x2])
    
    return xnew

A = np.array([[4,1,1],[1,2,3],[2,-1,-3]])
b = np.array([[5],[5],[-3]])
x = np.array([0,0,0])

print(jacobi(A,b,x))

[[1.25]
 [2.5 ]
 [1.  ]]


Modify your function so that:
- It carries out a series of iterations of the Jacobi method instead of just one iteration.
- It stops once it has either carried out the maximum number of iterations `niter`, or the iterations have converged within a specified tolerance `tol`, i.e. the **relative change** between successive iterations $\underline{\mathbf{x}}^{\left(k\right)}$ and $\underline{\mathbf{x}}^{\left(k+1\right)}$ is less than `tol` (just like in Lab 4):

$$
      \frac{\left\| \underline{\mathbf{x}}^{\left(k+1\right)}-\underline{\mathbf{x}}^{\left(k\right)}\right\|_\infty}
           {\left\|\underline{\mathbf{x}}^{\left(k+1\right)}\right\|_\infty}
        < \mathtt{tol}.
$$

- The maximum number of iterations and the acceptable tolerance are passed as inputs to the function.
- It returns a **matrix** of the results, the $k$th row of which contains $k$ and $\underline{\mathbf{x}}^{\left(k\right)}$, the $k$th approximation to the solution.

**Hint:** Refer to your multidimensional Newton's method function (Lab 4) as a model, particularly for the results table and convergence test. You function should look something like

```
def jacobi(A, b, x, niter, tol):
    ...
    return results
```

Try your function out using different values of `niter` and `tol` to make sure it is doing what it should do.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 1</h3>

  Print out the table returned by your function for <code>niter = 100</code> and <code>tol = 1e-6</code>.  Check that the solution from your program is correct.

  <b>Hint:</b> The results of the first 3 iterations after $\underline{\mathbf{x}}^{\left(0\right)}=(0,0,0)$ should be:
  <pre><code>
[[0.      1.25    2.5     1.     ]
 [1.      0.375   0.375   1.     ]
 [2.      0.90625 0.8125  1.125  ]]]
</code></pre>
</div>

In [11]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt

def jacobi(A,b,x, niter, tol):
    x = np.copy(x)
    
    #Loop to meet a tolerance or a max number of loops
    for i in range(niter+1):  
        x0 = ( b[0] - A[0][1]*x[1] - A[0][2]*x[2] ) / A[0][0]
        x1 = ( b[1] - A[1][0]*x[0] - A[1][2]*x[2] ) / A[1][1]
        x2 = ( b[2] - A[2][0]*x[0] - A[2][1]*x[1] ) / A[2][2]
        
        xnew = np.array([x0,x1,x2])
    
        highest_tol = 0
        
        
        #Finds the inifinty norm of (x(k+1)-x(k))/x(k+1)
        for i in range(3):
            diff = (xnew[i] - x[i])/xnew[i]
            if abs(diff) > highest_tol:
                highest_tol = abs(diff)
                
        if highest_tol < tol:
            return xnew
        
        x = np.copy(xnew)

A = np.array([[4,1,1],[1,2,3],[2,-1,-3]])
b = np.array([[5],[5],[-3]])
x = np.array([0,0,0])

print(jacobi(A,b,x, 100, 1e-6))

[[ 0.99999927]
 [-0.999995  ]
 [ 1.99999744]]


Now modify your function so that it also tells you whether or not convergence has occurred:
```
def jacobi(A, b, x, niter, tol):
    ...
    return results, converged
```
True/false information like this is usually conveyed using “flag” variables containing boolean values.  In this case:

- `converged = True` if the method converged in the specified number of iterations;
- `converged = False` if the method failed to converge in the specified number of iterations.


In [12]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt

def jacobi(A,b,x, niter, tol):
    x = np.copy(x)
    
    #Loop to meet a tolerance or a max number of loops
    for i in range(niter+1):  
        x0 = ( b[0] - A[0][1]*x[1] - A[0][2]*x[2] ) / A[0][0]
        x1 = ( b[1] - A[1][0]*x[0] - A[1][2]*x[2] ) / A[1][1]
        x2 = ( b[2] - A[2][0]*x[0] - A[2][1]*x[1] ) / A[2][2]
        
        xnew = np.array([x0,x1,x2])
    
        highest_tol = 0

        #Finds the inifinty norm of (x(k+1)-x(k))/x(k+1)
        for i in range(3):
            diff = (xnew[i] - x[i])/xnew[i]
            if abs(diff) > highest_tol:
                highest_tol = abs(diff)
                
        if highest_tol < tol:
            return xnew, True
        
        x = np.copy(xnew)
    
    return x, False

A = np.array([[4,1,1],[1,2,3],[2,-1,-3]])
b = np.array([[5],[5],[-3]])
x = np.array([0,0,0])

x, converged = jacobi(A,b,x, 100, 1e-6)
print(x)
print(converged)

[[ 0.99999927]
 [-0.999995  ]
 [ 1.99999744]]
True


Now write code that uses `jacobi` to solve the system of equations above, checks the value of the output flag, and displays an appropriate message, e.g.

```
"Iterative method converged to x = ... in ... iterations"
```
or
```
"Iterative method failed to converge in ... iterations"
```

where `...` should be replaced based on your function outputs `results` and `converged`.

There are a few different ways to do string interpolation in Python, such `str.format()`,

```
print('Iterative method converged to x = {} in {} iterations'.format(x, k + 1))
```

or f-string notation,

```
print(f'Iterative method converged to x = {x} in {k + 1} iterations')
```

You can format native Python types such as floats or integers using either method, see [Fancier Output Formatting](https://docs.python.org/3/tutorial/inputoutput.html#fancier-output-formatting) for an overview and [Format String Syntax](https://docs.python.org/3/library/string.html#format-string-syntax) for more detail. You can also simplify the object that you want to insert into the string **before** it is actually inserted,

```
print('Iterative method converged to x = {} in {} iterations'.format(x.round(4), k))
```

where `x` is rounded using `np.ndarray.round`.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 2</h3>

  Run your code twice, using different values of <code>niter</code> and <code>tol</code>, once where convergence is reached and once with convergence not happening.
</div>

In [27]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt

def jacobi(A,b,x, niter, tol):
    x = np.copy(x)
    
    #Loop to meet a tolerance or a max number of loops
    for i in range(niter+1):  
        x0 = ( b[0] - A[0][1]*x[1] - A[0][2]*x[2] ) / A[0][0]
        x1 = ( b[1] - A[1][0]*x[0] - A[1][2]*x[2] ) / A[1][1]
        x2 = ( b[2] - A[2][0]*x[0] - A[2][1]*x[1] ) / A[2][2]
        
        xnew = np.array([x0,x1,x2])
    
        highest_tol = 0
        
        #Finds the inifinty norm of (x(k+1)-x(k))/x(k+1)
        for a in range(3):
            diff = (xnew[a] - x[a])/xnew[a]
            if abs(diff) > highest_tol:
                highest_tol = abs(diff)
                
        if highest_tol < tol:
            return xnew, True, i
        
        x = np.copy(xnew)
    
    return x, False, i

A = np.array([[4,1,1],[1,2,3],[2,-1,-3]])
b = np.array([[5],[5],[-3]])
x = np.array([0,0,0])

x, converged, niter = jacobi(A,b,x, 100, 1e-6)
if converged == True:
    print(f"Iterative method converged to x = ({x[0][0]:.4F},{x[1][0]:.4F},{x[2][0]:.4F}) in {niter} iterations")
else:
    print(f"Iterative method failed to converge in {niter} iterations")

Iterative method converged to x = (1.0000,-1.0000,2.0000) in 75 iterations


In [29]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt

def jacobi(A,b,x, niter, tol):
    x = np.copy(x)
    
    #Loop to meet a tolerance or a max number of loops
    for i in range(niter+1):  
        x0 = ( b[0] - A[0][1]*x[1] - A[0][2]*x[2] ) / A[0][0]
        x1 = ( b[1] - A[1][0]*x[0] - A[1][2]*x[2] ) / A[1][1]
        x2 = ( b[2] - A[2][0]*x[0] - A[2][1]*x[1] ) / A[2][2]
        
        xnew = np.array([x0,x1,x2])
    
        highest_tol = 0
        
        #Finds the inifinty norm of (x(k+1)-x(k))/x(k+1)
        for a in range(3):
            diff = (xnew[a] - x[a])/xnew[a]
            if abs(diff) > highest_tol:
                highest_tol = abs(diff)
                
        if highest_tol < tol:
            return xnew, True, i
        
        x = np.copy(xnew)
    
    return x, False, i

A = np.array([[4,1,1],[1,2,3],[2,-1,-3]])
b = np.array([[5],[5],[-3]])
x = np.array([0,0,0])

x, converged, niter = jacobi(A,b,x, 10, 1e-6)
if converged == True:
    print(f"Iterative method converged to x = ({x[0][0]:.4F},{x[1][0]:.4F},{x[2][0]:.4F}) in {niter} iterations")
else:
    print(f"Iterative method failed to converge in {niter} iterations")

Iterative method failed to converge in 9 iterations


## Gauss–Seidel method

Write a function `gauss_seidel` that does the same thing as `jacobi`, but using the Gauss–Seidel method instead of the Jacobi method (you can use `jacobi` as a model so you don't have to start from scratch).

**Hint:** The first 3 iterations after $\underline{\mathbf{x}}^{\left(0\right)}=(0,0,0)$ should be:
```
[[0.         1.25       1.875      1.20833333]
 [1.         0.47916667 0.44791667 1.17013889]
 [2.         0.84548611 0.32204861 1.45630787]]
```
Write code that uses `gauss_seidel` to solve the system of equations above, checks the value of the output flag, and displays an appropriate message.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 3</h3>

  Run your code twice, using the same values of <code>niter</code> and <code>tol</code> that you used for the previous checkpoint.  Compare the Jacobi and Gauss–Seidel methods for this linear system; which is better?
</div>

In [37]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt

def jacobi(A,b,x, niter, tol):
    x = np.copy(x)
    
    #Loop to meet a tolerance or a max number of loops
    for i in range(niter+1):  
        x0 = ( b[0] - A[0][1]*x[1] - A[0][2]*x[2] ) / A[0][0]
        x1 = ( b[1] - A[1][0]*x0 - A[1][2]*x[2] ) / A[1][1]
        x2 = ( b[2] - A[2][0]*x0 - A[2][1]*x1 ) / A[2][2]
        
        xnew = np.array([x0,x1,x2])
    
        highest_tol = 0
        
        #Finds the inifinty norm of (x(k+1)-x(k))/x(k+1)
        for a in range(3):
            diff = (xnew[a] - x[a])/xnew[a]
            if abs(diff) > highest_tol:
                highest_tol = abs(diff)
                
        if highest_tol < tol:
            return xnew, True, i
        
        x = np.copy(xnew)
    
    return x, False, i

A = np.array([[4,1,1],[1,2,3],[2,-1,-3]])
b = np.array([[5],[5],[-3]])
x = np.array([0,0,0])

x, converged, niter = jacobi(A,b,x, 100, 1e-6)
if converged == True:
    print(f"Iterative method converged to x = ({x[0][0]:.4F},{x[1][0]:.4F},{x[2][0]:.4F}) in {niter} iterations")
else:
    print(f"Iterative method failed to converge in {niter} iterations")

Iterative method converged to x = (1.0000,-1.0000,2.0000) in 47 iterations


In [50]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt

def jacobi(A,b,x, niter, tol):
    x = np.copy(x)
    
    #Loop to meet a tolerance or a max number of loops
    for i in range(niter+1):  
        x0 = ( b[0] - A[0][1]*x[1] - A[0][2]*x[2] ) / A[0][0]
        x1 = ( b[1] - A[1][0]*x0 - A[1][2]*x[2] ) / A[1][1]
        x2 = ( b[2] - A[2][0]*x0 - A[2][1]*x1 ) / A[2][2]
        
        xnew = np.array([x0,x1,x2])
    
        highest_tol = 0
        
        #Finds the inifinty norm of (x(k+1)-x(k))/x(k+1)
        for a in range(3):
            diff = (xnew[a] - x[a])/xnew[a]
            if abs(diff) > highest_tol:
                highest_tol = abs(diff)
                
        if highest_tol < tol:
            return xnew, True, i
        
        x = np.copy(xnew)
    
    return x, False, i

A = np.array([[4,1,1],[1,2,3],[2,-1,-3]])
b = np.array([[5],[5],[-3]])
x = np.array([0,0,0])

x, converged, niter = jacobi(A,b,x, 20, 1e-6)
if converged == True:
    print(f"Iterative method converged to x = ({x[0][0]:.4F},{x[1][0]:.4F},{x[2][0]:.4F}) in {niter} iterations")
else:
    print(f"Iterative method failed to converge in {niter} iterations")
    if abs(x).max() > 10e9:
        print("x likely has diverged")

Iterative method failed to converge in 20 iterations


In [ ]:
type your code here

Now write code that uses your modified `gauss_seidel` to solve the following systems of equations.

$$
  \textrm{(i)} \qquad A = \begin{bmatrix} \phantom{-}4 & -2 & \phantom{-}1 \\ \phantom{-}3 & -6 & \phantom{-}1 \\ -4 & \phantom{-}1 & \phantom{-}6 \end{bmatrix} \quad
  \underline{\mathbf{b}} = \begin{bmatrix}
          \phantom{-}5 \\
          \phantom{-}5 \\
          -3
        \end{bmatrix},
  \qquad
  \textrm{(ii)} \qquad A = \begin{bmatrix} \phantom{-}1 & -2 & \phantom{-}1 \\ \phantom{-}3 & \phantom{-}1 & \phantom{-}1 \\ -4 & \phantom{-}1 & \phantom{-}1\end{bmatrix} \quad
  \underline{\mathbf{b}} = \begin{bmatrix}
          \phantom{-}5 \\
          \phantom{-}5 \\
          -3
        \end{bmatrix}.
$$

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 4</h3>

  What happened and why? Check your solutions are correct using <code>np.linalg.solve</code>.
</div>

In [41]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt

def jacobi(A,b,x, niter, tol):
    x = np.copy(x)
    
    #Loop to meet a tolerance or a max number of loops
    for i in range(niter+1):  
        x0 = ( b[0] - A[0][1]*x[1] - A[0][2]*x[2] ) / A[0][0]
        x1 = ( b[1] - A[1][0]*x0 - A[1][2]*x[2] ) / A[1][1]
        x2 = ( b[2] - A[2][0]*x0 - A[2][1]*x1 ) / A[2][2]
        
        xnew = np.array([x0,x1,x2])
    
        highest_tol = 0
        
        #Finds the inifinty norm of (x(k+1)-x(k))/x(k+1)
        for a in range(3):
            diff = (xnew[a] - x[a])/xnew[a]
            if abs(diff) > highest_tol:
                highest_tol = abs(diff)
                
        if highest_tol < tol:
            return xnew, True, i
        
        x = np.copy(xnew)
    
    return x, False, i

A = np.array([[4,-2,1],[3,-6,1],[-4,1,6]])
b = np.array([[5],[5],[-3]])
x = np.array([0,0,0])

x_linalg = np.linalg.solve(A,b)

print(f"x via linalg; ({x_linalg[0][0]:.4F},{x_linalg[1][0]:.4F},{x_linalg[2][0]:.4F})")


x, converged, niter = jacobi(A,b,x, 100, 1e-6)
if converged == True:
    print(f"Iterative method converged to x = ({x[0][0]:.4F},{x[1][0]:.4F},{x[2][0]:.4F}) in {niter} iterations")
else:
    print(f"Iterative method failed to converge in {niter} iterations")
    if abs(x).max() > 10e9:
        print("x likely has diverged")

x via linalg; (1.0560,-0.2640,0.2480)
Iterative method converged to x = (1.0560,-0.2640,0.2480) in 11 iterations


In [49]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt

def jacobi(A,b,x, niter, tol):
    x = np.copy(x)
    
    #Loop to meet a tolerance or a max number of loops
    for i in range(niter+1):  
        x0 = ( b[0] - A[0][1]*x[1] - A[0][2]*x[2] ) / A[0][0]
        x1 = ( b[1] - A[1][0]*x0 - A[1][2]*x[2] ) / A[1][1]
        x2 = ( b[2] - A[2][0]*x0 - A[2][1]*x1 ) / A[2][2]
        
        xnew = np.array([x0,x1,x2])
    
        highest_tol = 0
        
        #Finds the inifinty norm of (x(k+1)-x(k))/x(k+1)
        for a in range(3):
            diff = (xnew[a] - x[a])/xnew[a]
            if abs(diff) > highest_tol:
                highest_tol = abs(diff)
                
        if highest_tol < tol:
            return xnew, True, i
        
        x = np.copy(xnew)
    return x, False, i

A = np.array([[1,-2,1],[3,1,1],[-4,1,1]])
b = np.array([[5],[5],[-3]])
x = np.array([0,0,0])

print(f"x via linalg; ({x_linalg[0][0]:.4F},{x_linalg[1][0]:.4F},{x_linalg[2][0]:.4F})")

x, converged, niter = jacobi(A,b,x, 100, 1e-6)
if converged == True:
    print(f"Iterative method converged to x = ({x[0][0]:.4F},{x[1][0]:.4F},{x[2][0]:.4F}) in {niter} iterations")
else:
    print(f"Iterative method failed to converge in {niter} iterations")
    if abs(x).max() > 10e9:
        print("x likely has diverged")

x via linalg; (1.0560,-0.2640,0.2480)
Iterative method failed to converge in 100 iterations
x likely has diverged
